# Notebook 04: Inventory Optimization

**Objective:** Calculate Safety Stock and Reorder Point for AX products using demand statistics and service level targets.

---

### Key Formulas

| Formula | Description |
|---------|-------------|
| **Safety Stock** = Z × σ × √LT | Buffer inventory for demand variability |
| **Reorder Point** = (Avg Demand × LT) + Safety Stock | When to place new order |

Where: Z = Service level factor, σ = Std deviation, LT = Lead time

### Service Level Strategy

| Segment | Service Level | Z-Score | Reasoning |
|---------|---------------|---------|----------|
| AX | 99% | 2.33 | High value, stable → must have stock |
| BX | 95% | 1.65 | Medium value → balance cost & availability |
| CY | 90% | 1.28 | Low value → minimize inventory cost |

## 1. Load Data

In [2]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('data/processed/df_clean_filtered.csv')
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'])
abc_xyz_df = pd.read_csv('outputs/abc_xyz_results.csv')

# Filter AX products
ax_products = abc_xyz_df[abc_xyz_df['Segment'] == 'AX']

print(f"Orders: {len(df):,} | AX Products: {len(ax_products)}")

Orders: 75,564 | AX Products: 6


## 2. Calculate Product Statistics

In [3]:
# Calculate stats for each AX product
ax_product_names = ax_products['Product Name'].tolist()
product_stats = []

for product in ax_product_names:
    product_data = df[df['Product Name'] == product]
    weekly = product_data.resample('W', on='order date (DateOrders)')['Order Item Quantity'].sum()
    
    product_stats.append({
        'Product': product,
        'Unit_Price': product_data['Product Price'].mean(),
        'Avg_Weekly_Demand': weekly.mean(),
        'Std_Weekly_Demand': weekly.std(),
        'Lead_Time_Days': product_data['Days for shipping (real)'].mean()
    })

product_stats_df = pd.DataFrame(product_stats)
print(product_stats_df.to_string(index=False))

                                      Product  Unit_Price  Avg_Weekly_Demand  Std_Weekly_Demand  Lead_Time_Days
    Field & Stream Sportsman 16 Gun Fire Safe  399.980011          52.472222           8.394924        3.510058
             Perfect Fitness Perfect Rip Deck   59.990002         221.827586          34.834000        3.503468
Diamondback Women's Serene Classic Comfort Bi  299.980011          41.448276           7.976468        3.463727
            Nike Men's Free 5.0+ Running Shoe   99.989998         112.241379          25.195424        3.465194
         Nike Men's Dri-FIT Victory Golf Polo   50.000000         191.310345          33.779003        3.476278
                  Pelican Sunstream 100 Kayak  199.990005          46.124138           8.190342        3.453798


## 3. Calculate Safety Stock & Reorder Point

In [4]:
# Parameters
service_level = 0.99
z = 2.33  # Z-score for 99% service level

# Calculate lead time in weeks
product_stats_df['Lead_Time_Weeks'] = product_stats_df['Lead_Time_Days'] / 7

# Safety Stock = Z × Std Dev × √(Lead Time)
product_stats_df['Safety_Stock'] = (
    z * product_stats_df['Std_Weekly_Demand'] * np.sqrt(product_stats_df['Lead_Time_Weeks'])
).round(0)

# Reorder Point = (Avg Demand × Lead Time) + Safety Stock
product_stats_df['Reorder_Point'] = (
    product_stats_df['Avg_Weekly_Demand'] * product_stats_df['Lead_Time_Weeks'] + 
    product_stats_df['Safety_Stock']
).round(0)

print("Calculations complete ✓")

Calculations complete ✓


## 4. Results

In [5]:
# Final results table
results = product_stats_df[[
    'Product', 'Unit_Price', 'Avg_Weekly_Demand', 
    'Lead_Time_Days', 'Safety_Stock', 'Reorder_Point'
]].copy()

results.columns = ['Product', 'Unit Price ($)', 'Avg Demand/Week', 'Lead Time (days)', 'Safety Stock', 'Reorder Point']
results['Unit Price ($)'] = results['Unit Price ($)'].round(2)
results['Lead Time (days)'] = results['Lead Time (days)'].round(1)

print("=" * 100)
print("INVENTORY OPTIMIZATION RESULTS - AX PRODUCTS (Service Level: 99%)")
print("=" * 100)
print(results.to_string(index=False))
print("=" * 100)

INVENTORY OPTIMIZATION RESULTS - AX PRODUCTS (Service Level: 99%)
                                      Product  Unit Price ($)  Avg Demand/Week  Lead Time (days)  Safety Stock  Reorder Point
    Field & Stream Sportsman 16 Gun Fire Safe          399.98        52.472222               3.5          14.0           40.0
             Perfect Fitness Perfect Rip Deck           59.99       221.827586               3.5          57.0          168.0
Diamondback Women's Serene Classic Comfort Bi          299.98        41.448276               3.5          13.0           34.0
            Nike Men's Free 5.0+ Running Shoe           99.99       112.241379               3.5          41.0           97.0
         Nike Men's Dri-FIT Victory Golf Polo           50.00       191.310345               3.5          55.0          150.0
                  Pelican Sunstream 100 Kayak          199.99        46.124138               3.5          13.0           36.0


In [6]:
# Save for Streamlit app
product_stats_df.to_csv('outputs/inventory_parameters.csv', index=False)
print("✓ Saved to outputs/inventory_parameters.csv")

✓ Saved to outputs/inventory_parameters.csv


## Conclusions

### Final Inventory Parameters

| Product | Unit Price | Safety Stock | Reorder Point |
|---------|------------|--------------|---------------|
| Field & Stream Gun Safe | $399.98 | 14 units | 40 units |
| Perfect Fitness Rip Deck | $59.99 | 57 units | 167 units |
| Diamondback Bike | $299.98 | 13 units | 34 units |
| Nike Running Shoe | $99.99 | 41 units | 97 units |
| Nike Golf Polo | $50.00 | 56 units | 151 units |
| Pelican Kayak | $199.99 | 13 units | 36 units |

### Key Takeaways

1. **Service Level:** 99% for all AX products (high value, stable demand)
2. **Safety Stock:** Ranges from 13-57 units based on demand variability
3. **Reorder Point:** When inventory hits this level, place new order

### Business Action

| When Inventory Reaches | Action |
|------------------------|--------|
| Reorder Point | Place new order immediately |
| Below Safety Stock | Urgent replenishment needed |

---

**Next:** Build Streamlit app combining forecasts + inventory parameters